<a href="https://colab.research.google.com/github/shivansh2310/Finance-Stuff/blob/main/Trinomial_Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def calculate_strike_prices(S0, moneyness_levels):
    strike_prices = []
    for moneyness in moneyness_levels:
        K = S0 * moneyness
        strike_prices.append(K)
    return strike_prices

# Initial stock price
S0 = 100

# Moneyness levels for Deep OTM, OTM, ATM, ITM, Deep ITM
moneyness_levels = [0.9, 0.95, 1.0, 1.05, 1.1]

# Calculate strike prices
strike_prices = calculate_strike_prices(S0, moneyness_levels)

# Print the calculated strike prices with moneyness types
for i, K in enumerate(strike_prices):
    moneyness_type = ""
    if i == 0:
        moneyness_type = "Deep OTM"
    elif i == 1:
        moneyness_type = "OTM"
    elif i == 2:
        moneyness_type = "ATM"
    elif i == 3:
        moneyness_type = "ITM"
    elif i == 4:
        moneyness_type = "Deep ITM"

    print(f"{moneyness_type}: Strike Price = {K:.2f}")


Deep OTM: Strike Price = 90.00
OTM: Strike Price = 95.00
ATM: Strike Price = 100.00
ITM: Strike Price = 105.00
Deep ITM: Strike Price = 110.00


In [3]:
import numpy as np


In [4]:
def _gen_stock_vec(nb, h):
    s0 = 100
    sigma = 0.5
    up = np.exp(sigma * np.sqrt(2 * h))
    down = 1 / up  # down movement to force a "recombining tree"

    vec_u = up * np.ones(nb)
    np.cumprod(vec_u, out=vec_u)  # Computing u, u^2, u^3....u^nb

    vec_d = down * np.ones(nb)
    np.cumprod(vec_d, out=vec_d)  # Computing d, d^2, d^3....d^nb

    res = np.concatenate(
        (vec_d[::-1], [1.0], vec_u)
    )  # putting together the last period tree underlyings
    res *= s0
    return res

In [5]:
def price(
    nb_steps,
):  # For now, the only input to the function is the number of steps for the tree, N.
    # Define parameters
    r = 0
    sigma = 0.5
    T = 1
    K = 130
    h = T / nb_steps  # This would be our 'dt' from previous examples
    discount = np.exp(-r * h)  # Define discount factor for simplicity later on

    # Define risk-neutral probabilities:
    pu = (
        (np.exp(r * h / 2) - np.exp(-sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pd = (
        (-np.exp(r * h / 2) + np.exp(sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pm = 1 - pu - pd
    print(pu, pd, pm)
    # This would be our underlying evolution (Note we are using the function from before!)
    s = _gen_stock_vec(nb_steps, h)

    # Define Payoff (in this case, European Call Option)
    final_payoff = np.maximum(s - K, 0)
    nxt_vec_prices = final_payoff

    # Proceed with iterations for the calculation of payoffs
    for i in range(1, nb_steps + 1):
        vec_stock = _gen_stock_vec(nb_steps - i, h)
        expectation = np.zeros(vec_stock.size)

        for j in range(expectation.size):
            tmp = nxt_vec_prices[j] * pd
            tmp += nxt_vec_prices[j + 1] * pm
            tmp += nxt_vec_prices[j + 2] * pu

            expectation[j] = tmp
        # Discount option payoff!
        nxt_vec_prices = discount * expectation

    return nxt_vec_prices[0]

In [6]:
print(" Price of the Euro Call Option is ", price(100))

0.24124019455888882 0.25891602289479126 0.49984378254631995
 Price of the Euro Call Option is  10.686959567311135


In [9]:
def price_put(
    nb_steps,
):  # For now, the only input to the function is the number of steps for the tree, N.
    # Define parameters
    r = 0
    sigma = 0.5
    T = 1
    K = 110
    h = T / nb_steps  # This would be our 'dt' from previous examples
    discount = np.exp(-r * h)  # Define discount factor for simplicity later on

    # Define risk-neutral probabilities:
    pu = (
        (np.exp(r * h / 2) - np.exp(-sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pd = (
        (-np.exp(r * h / 2) + np.exp(sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pm = 1 - pu - pd
    #print(pu, pd, pm)
    # This would be our underlying evolution (Note we are using the function from before!)
    s = _gen_stock_vec(nb_steps, h)

    # Define Payoff (in this case, European Call Option)
    final_payoff = np.maximum(K - s, 0)
    nxt_vec_prices = final_payoff

    # Proceed with iterations for the calculation of payoffs
    for i in range(1, nb_steps + 1):
        vec_stock = _gen_stock_vec(nb_steps - i, h)
        expectation = np.zeros(vec_stock.size)

        for j in range(expectation.size):
            tmp = nxt_vec_prices[j] * pd
            tmp += nxt_vec_prices[j + 1] * pm
            tmp += nxt_vec_prices[j + 2] * pu

            expectation[j] = tmp
        # Discount option payoff!
        nxt_vec_prices = discount * expectation

    return nxt_vec_prices[
        0
    ]

In [10]:
print(" Price of the Euro Put Option is ", price_put(100))

 Price of the Euro Put Option is  26.115031285940894


In [17]:
import numpy as np

def price_american(nb_steps, option_type):
    # Define parameters
    r = 0  # Risk-free rate
    sigma = 0.5  # Volatility
    T = 1  # Time to expiration
    K = 130  # Strike price
    h = T / nb_steps  # Time step size (dt)
    discount = np.exp(-r * h)  # Discount factor for simplicity later on

    # Risk-neutral probabilities
    pu = ((np.exp(r * h / 2) - np.exp(-sigma * np.sqrt(h / 2))) /
          (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))) ** 2
    pd = ((-np.exp(r * h / 2) + np.exp(sigma * np.sqrt(h / 2))) /
          (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))) ** 2
    pm = 1 - pu - pd

    # Generate stock price vector
    s = _gen_stock_vec(nb_steps, h)

    # Define Payoff (European Call/Put Option)
    if option_type == 'call':
        payoff = np.maximum(s - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - s, 0)

    nxt_vec_prices = payoff  # Initialize option prices with payoff

    # Iterate backward for the calculation of option prices
    for i in range(1, nb_steps + 1):
        vec_stock = _gen_stock_vec(nb_steps - i, h)
        expectation = np.zeros(vec_stock.size)

        for j in range(expectation.size):
            tmp = nxt_vec_prices[j] * pd
            tmp += nxt_vec_prices[j + 1] * pm
            tmp += nxt_vec_prices[j + 2] * pu

            if option_type == 'call':
                # Early exercise check for American Call Option
                early_exercise = np.maximum(s[j] - K, 0)
            elif option_type == 'put':
                # Early exercise check for American Put Option
                early_exercise = np.maximum(K - s[j], 0)

            expectation[j] = max(tmp, early_exercise)  # Compare with early exercise

        nxt_vec_prices = discount * expectation  # Discount option payoff

    return nxt_vec_prices[0]

# Example usage
american_call_price = price_american(90, option_type='call')
american_put_price = price_american(90, option_type='put')

print("American Call Option Price:", american_call_price)
print("American Put Option Price:", american_put_price)


American Call Option Price: 10.69740209547745
American Put Option Price: 129.87791451055736


In [66]:
class TrinomialModel(object):  # Here we start defining our 'class' --> Trinomial Model!
    # First, a method to initialize our `TrinomialModel` algorithm!
    def __init__(self, S0, r, sigma, mat):
        self.__s0 = S0
        self.__r = r
        self.__sigma = sigma
        self.__T = mat

    # Second, we build a method (function) to compute the risk-neutral probabilities!
    def __compute_probs(self):
        self.__pu = (
            (
                np.exp(self.__r * self.__h / 2)
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
            / (
                np.exp(self.__sigma * np.sqrt(self.__h / 2))
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
        ) ** 2
        self.__pd = (
            (
                -np.exp(self.__r * self.__h / 2)
                + np.exp(self.__sigma * np.sqrt(self.__h / 2))
            )
            / (
                np.exp(self.__sigma * np.sqrt(self.__h / 2))
                - np.exp(-self.__sigma * np.sqrt(self.__h / 2))
            )
        ) ** 2
        self.__pm = 1 - self.__pu - self.__pd

        assert 0 <= self.__pu <= 1.0, "p_u should lie in [0, 1] given %s" % self.__pu
        assert 0 <= self.__pd <= 1.0, "p_d should lie in [0, 1] given %s" % self.__pd
        assert 0 <= self.__pm <= 1.0, "p_m should lie in [0, 1] given %s" % self.__pm

    # Third, this method checks whether the given parameters are alright and that we have a 'recombining tree'!
    def __check_up_value(self, up):
        if up is None:
            up = np.exp(self.__sigma * np.sqrt(2 * self.__h))

        assert up > 0.0, "up should be non negative"

        down = 1 / up

        assert down < up, "up <= 1. / up = down"

        self.__up = up
        self.__down = down

    # Four, we use this method to compute underlying stock price path
    def __gen_stock_vec(self, nb):
        vec_u = self.__up * np.ones(nb)
        np.cumprod(vec_u, out=vec_u)

        vec_d = self.__down * np.ones(nb)
        np.cumprod(vec_d, out=vec_d)

        res = np.concatenate((vec_d[::-1], [1.0], vec_u))
        res *= self.__s0

        return res

    # Fifth, we declare a Payoff method to be completed afterwards depending on the instrument we are pricing!
    def payoff(self, stock_vec):
        raise NotImplementedError()

    # Sixth, compute current prices!
    def compute_current_price(self, crt_vec_stock, nxt_vec_prices):
        expectation = np.zeros(crt_vec_stock.size)
        for i in range(expectation.size):
            tmp = nxt_vec_prices[i] * self.__pd
            tmp += nxt_vec_prices[i + 1] * self.__pm
            tmp += nxt_vec_prices[i + 2] * self.__pu

            expectation[i] = tmp

        return self.__discount * expectation

    # Seventh, Option pricing!
    def price(self, nb_steps, up=None):
        assert nb_steps > 0, "nb_steps shoud be > 0"

        nb_steps = int(nb_steps)

        self.__h = self.__T / nb_steps
        self.__check_up_value(up)
        self.__compute_probs()

        self.__discount = np.exp(-self.__r * self.__h)

        final_vec_stock = self.__gen_stock_vec(nb_steps)
        final_payoff = self.payoff(final_vec_stock)
        nxt_vec_prices = final_payoff

        for i in range(1, nb_steps + 1):
            vec_stock = self.__gen_stock_vec(nb_steps - i)
            nxt_vec_prices = self.compute_current_price(vec_stock, nxt_vec_prices)

        return nxt_vec_prices[0]

In [67]:
class TrinomialCall(TrinomialModel):
    def __init__(self, S0, r, sigma, mat, K):
        super(TrinomialCall, self).__init__(S0, r, sigma, mat)
        self.__K = K

    def payoff(self, s):
        return np.maximum(s - self.__K, 0.0)

In [68]:
tree = TrinomialCall(100.0, 0.0, 0.3, 1.0, 90.0)

In [69]:
print(tree.price(2))

17.50162310051333
